In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

<h3>Create a dataset with nurse data and frailty index for training the rf</h3>

Load all the nurse datasets

In [2]:
df1 = pd.read_csv (r'C:\Users\Manos\OneDrive\My_thesis\tab_per_wave\Wave_2\wave_2_nurse_data_v2.tab',sep='\t',low_memory=False)
df2 = pd.read_csv (r'C:\Users\Manos\OneDrive\My_thesis\tab_per_wave\Wave_4\wave_4_nurse_data.tab',sep='\t',low_memory=False)
df3 = pd.read_csv (r'C:\Users\Manos\OneDrive\My_thesis\tab_per_wave\Wave_6\wave_6_elsa_nurse_data_v2.tab',sep='\t',low_memory=False)
df4 = pd.read_csv (r'C:\Users\Manos\OneDrive\My_thesis\tab_per_wave\Wave_9\elsa_nurse_w8w9_data_eul.tab',sep='\t',low_memory=False)

create a dictionray for ease fo use

In [3]:
dicto={1:df1,
      2:df2,
      3:df3,
      4:df4
     }

Keep only the nurse ralated col

In [4]:
feat_list=['idauniq','sys1','sys2','sys3','dias1','dias2','dias3','pulse1','pulse2','pulse3','cfib',
       'chol','hdl','trig','ldl','rtin','hscrp','hgb','hba1c','mmgsd1','mmgsd2','mmgsd3',
       'mmgsn1','mmgsn2','mmgsn3']

In [5]:
for i in range (1,5):
    dicto[i]=dicto[i][feat_list]

Load frailty indexes for waves 2,4,6,8 and keep only idauniq and fi

In [6]:
fdf1 = pd.read_csv (r'C:\Users\Manos\OneDrive\My_thesis\prepared_data\Frailty_Indices\Frailty_Index_Wave_2.tab',sep='\t',low_memory=False)
fdf2 = pd.read_csv (r'C:\Users\Manos\OneDrive\My_thesis\prepared_data\Frailty_Indices\Frailty_Index_Wave_4.tab',sep='\t',low_memory=False)
fdf3 = pd.read_csv (r'C:\Users\Manos\OneDrive\My_thesis\prepared_data\Frailty_Indices\Frailty_Index_Wave_6.tab',sep='\t',low_memory=False)
fdf4 = pd.read_csv (r'C:\Users\Manos\OneDrive\My_thesis\prepared_data\Frailty_Indices\Frailty_Index_Wave_8.tab',sep='\t',low_memory=False)

In [7]:
fdicto={1:fdf1,
      2:fdf2,
      3:fdf3,
      4:fdf4
     }

In [8]:
for i in range (1,5):
    fdicto[i]=fdicto[i][['idauniq','FI']]

Merge fi with nurse

In [9]:
for i in range (1,5):
    dicto[i] = pd.merge(dicto[i], fdicto[i], how='inner', on=['idauniq'])

Concat frames to create a set for training

In [10]:
frames=[dicto[1],dicto[2],dicto[3],dicto[4]]
result=pd.concat(frames)

<h4>KNN Imputation</h4>

convert object dtypes to floats 

In [11]:
types=result.dtypes
#types

In [12]:
#get the indexes of values equal to object and then convert them to a list with .tolist() function
list_obj=types[types=='object'].index.tolist()

In [13]:
for i in range (0,len(list_obj)):
    result[list_obj[i]]=pd.to_numeric(result[list_obj[i]], errors='coerce')

In [14]:
types=result.dtypes
#types

replace mssing values with nan

In [15]:
for k in range(-12,0):
  result=result.replace(k,np.nan)

In [16]:
percent_missing = result.isnull().sum() * 100 / len(result)
missing_value_df = pd.DataFrame({'column_name': result.columns,
                                 'percent_missing': percent_missing})

In [17]:
missing_value_df

,column_name,percent_missing
idauniq,idauniq,0.000000
sys1,sys1,0.618481
sys2,sys2,0.725461
sys3,sys3,0.842471
dias1,dias1,0.621824
dias2,dias2,0.722118
dias3,dias3,0.842471
pulse1,pulse1,0.628510
pulse2,pulse2,0.715432
pulse3,pulse3,0.845814


In [18]:
#define knn
from sklearn.impute import KNNImputer
knnimputer= KNNImputer(n_neighbors=5)
#apply knn
imputed=knnimputer.fit_transform(result)

In [19]:
feat_list=['idauniq','sys1','sys2','sys3','dias1','dias2','dias3','pulse1','pulse2','pulse3','cfib',
       'chol','hdl','trig','ldl','rtin','hscrp','hgb','hba1c','mmgsd1','mmgsd2','mmgsd3',
       'mmgsn1','mmgsn2','mmgsn3','FI']
imputed = pd.DataFrame(imputed, columns = feat_list)

Export for Future use

In [20]:
# imputed.to_csv(r'C:\Users\Manos\OneDrive/Thesis Final/LSTM/all_nurse_with_FI_imputed.tab',sep='\t',index=False)

In [21]:
imputed=pd.read_csv (r'C:\Users\Manos\OneDrive/Thesis Final/LSTM/all_nurse_with_FI_imputed.tab',sep='\t',low_memory=False)

In [22]:
imputed.head()

,idauniq,sys1,sys2,sys3,dias1,dias2,dias3,pulse1,pulse2,pulse3,...,hscrp,hgb,hba1c,mmgsd1,mmgsd2,mmgsd3,mmgsn1,mmgsn2,mmgsn3,FI
0,103712.0,152.0,145.0,150.0,84.0,80.0,78.0,78.0,77.0,79.0,...,0.5,13.8,6.8,20.0,24.0,26.0,30.0,31.0,30.0,0.150938
1,103713.0,143.0,141.0,144.0,91.0,86.0,85.0,75.0,78.0,76.0,...,3.3,11.2,5.6,29.4,29.8,30.0,33.0,38.0,35.0,0.270625
2,103714.0,150.0,143.0,139.0,91.0,100.0,91.0,86.0,86.0,81.0,...,0.8,15.0,4.9,28.0,29.0,27.0,30.0,31.0,30.0,0.046875
3,103715.0,142.0,138.0,144.0,78.0,78.0,74.0,68.0,69.0,69.0,...,3.0,15.3,5.4,32.0,30.0,30.0,26.0,32.0,30.0,0.078125
4,103716.0,160.0,159.0,158.0,82.0,81.0,81.0,56.0,54.0,55.0,...,1.1,15.1,5.3,26.0,24.0,23.0,29.0,28.0,29.0,0.117188


Random Forest

split dataset to indepent and target variables

In [23]:
X=imputed.iloc[:,1:-1]
y=imputed.iloc[:,-1]

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, random_state=32)

In [25]:
rf= RandomForestRegressor(n_estimators = 200)
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=200)

Load the df that has been predicted from the LSTM

In [26]:
# pred = pd.read_csv (r'C:\Users\Manos/OneDrive/Thesis Final/LSTM/Scaled_and_unidemensiolized_datasets/predicted_wave8.tab',sep='\t',low_memory=False)

make FI prediction

In [27]:
y_pred=rf.predict(X_test)#dont include idauniq

In [28]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error,mean_squared_error
mse=mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
r2=r2_score(y_test, y_pred)
adjr2=1-(((1-r2)*(len(y)-1))/((len(y)-X.shape[1]-1)))
print('MSE: %.3f' % mse)
print('MAE: %.3f' % mae)
print('MAPE: %.3f' % mape)
print('r2: %.3f' % r2)
print('adjr2: %.3f' % adjr2)

MSE: 0.012
MAE: 0.082
MAPE: 23724039178999.074
r2: 0.247
adjr2: 0.247


In [29]:
MSE: 0.012
MAE: 0.082
MAPE: 0.698
r2: -1.741
adjr2: -1.743

In [30]:
LSTM_prediction=pd.read_csv (r'C:\Users\Manos\Desktop\Lstm Prediction',sep='\t',low_memory=False)

In [31]:
FI_from_LSTM=rf.predict(LSTM_prediction.iloc[:,1:])#

In [32]:
FI_from_LSTM= pd.DataFrame(FI_from_LSTM, columns = ['FI'])
FI_from_LSTM.insert(0,'idauniq',LSTM_prediction.iloc[:,0])
FI_from_LSTM.to_csv(r'C:\Users\Manos\Desktop\RF_FI_LSTM',sep='\t',index=False)